Initialize Jupyter Notebook

In [10]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from web3 import Web3
import json
import time

# Load environment variables
load_dotenv()

# Connect to the Base network
w3 = Web3(Web3.HTTPProvider(os.getenv('RPC_URL')))

# Load contract ABIs
# Note: You'll need to create/update these ABI files with the new contract ABIs
with open('AffiliateERC1155StorefrontFactory_ABI.json', 'r') as abi_file:
    storefront_factory_abi = json.load(abi_file)
    
with open('AffiliateERC1155Storefront_ABI.json', 'r') as abi_file:
    storefront_abi = json.load(abi_file)
    
with open('ReceiptERC1155Factory_ABI.json', 'r') as abi_file:
    receipt_factory_abi = json.load(abi_file)
    
with open('ReceiptERC1155_ABI.json', 'r') as abi_file:
    receipt_erc1155_abi = json.load(abi_file)
    
with open('ERC20_ABI.json', 'r') as abi_file:
    erc20_abi = json.load(abi_file)
    
with open('AffiliateEscrow_ABI.json', 'r') as abi_file:
    affiliate_escrow_abi = json.load(abi_file)
    
with open('UmpMetadata_ABI.json', 'r') as abi_file:
    ump_metadata_abi = json.load(abi_file)

with open('seaport_abi.json', 'r') as abi_file:
    seaport_abi = json.load(abi_file)

# Set up contract addresses
UMP_METADATA_CONTRACT = os.getenv('UMP_METADATA_ADDRESS')
ump_metadata_contract = w3.eth.contract(address=UMP_METADATA_CONTRACT, abi=ump_metadata_abi)

# New contract addresses from deployment
AFFILIATE_STOREFRONT_FACTORY_ADDRESS = "0x59321B6B3AA8573DD33A6314cb48C6849Cd55C72"
AFFILIATE_VERIFIER_PROXY_ADDRESS = "0x1D1D4681E5146A66A9ff6917Ef068af5254617F6"
AFFILIATE_ESCROW_FACTORY_ADDRESS = "0xE07c41Bc76A8B56ad7E996cF60A3dDeD96ca575D"
RECEIPT_FACTORY_ADDRESS = "0x5168980B306923C9911A5A9b541254862e91683d"

# Set up contract instances
storefront_factory_contract = w3.eth.contract(
    address=AFFILIATE_STOREFRONT_FACTORY_ADDRESS, 
    abi=storefront_factory_abi
)

receipt_factory_contract = w3.eth.contract(
    address=RECEIPT_FACTORY_ADDRESS, 
    abi=receipt_factory_abi
)

# Set up verifier contract
with open('AffiliateVerifier_ABI.json', 'r') as abi_file:
    affiliate_verifier_abi = json.load(abi_file)
    
affiliate_verifier_contract = w3.eth.contract(
    address=AFFILIATE_VERIFIER_PROXY_ADDRESS,
    abi=affiliate_verifier_abi
)

# Set up accounts
seller_account = w3.eth.account.from_key(os.getenv('SELLER_PRIVATE_KEY'))
owner_account = w3.eth.account.from_key(os.getenv('OWNER_PRIVATE_KEY'))

# Constants for the affiliate system
AFFILIATE_FEE_DENOMINATOR = 10000  # Base points, 100% = 10000

First the seller creates their own receipt ERC-1155 contract

In [11]:

def create_receipt_erc1155(contract_uri, factory_contract, owner_account):
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = factory_contract.functions.createReceiptERC1155(contract_uri).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Transaction receipt: {tx_receipt}")
    
    # Process all logs in the receipt
    for log in tx_receipt['logs']:
        print(f"Log: {log}")
        try:
            event = factory_contract.events.ReceiptERC1155Created().process_log(log)
            if event:
                return event['args']['tokenAddress']
        except Exception as e:
            print(f"Error processing log: {e}")
    
    print("No ReceiptERC1155Created event found in the transaction logs")
    return None


# Example usage
contract_uri = json.dumps({
    "name": "Wassie Hot Sauce Lab",
    "description": "Wassies experimenting on new contracts. You can make a test purchase here to try out our XMTP integration and CollateralEscrow",
    "image": "ipfs://bafybeidtxa4nu7wsxlux67vygsa3md3qyv6fq4rdfui5shuufgl6at3kbe",
    "external_link": "https://ump.eth.limo/"
 })

new_erc1155_address = create_receipt_erc1155(contract_uri, receipt_factory_contract, owner_account)

Transaction receipt: AttributeDict({'blockHash': HexBytes('0xb05dfe2cc89332be10954988f3a1e45d3a954d2f9032b02e46a2fc764f7c7310'), 'blockNumber': 27246978, 'contractAddress': None, 'cumulativeGasUsed': 50729813, 'effectiveGasPrice': 2434840, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 2355926, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0x4ef9325c', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x6b9234cea6', 'l1GasPrice': '0x8c6c4784', 'l1GasUsed': '0x1374', 'logs': [AttributeDict({'address': '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 'blockHash': HexBytes('0xb05dfe2cc89332be10954988f3a1e45d3a954d2f9032b02e46a2fc764f7c7310'), 'blockNumber': 27246978, 'data': HexBytes('0x'), 'logIndex': 437, 'removed': False, 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x0000000000000000000000005168980b306923c9911a5a9b541254862e91683d')], 'trans

TODO: figure out why the event signature isnt matching the abi (low priority)

# Seller mints the NFTs they want to sell
## Seller sets metadata on the NFTs

In [12]:

def mint_erc1155_tokens(contract_address, token_id, amount, recipient_address, owner_account):
    # Load the ABI for the ReceiptERC1155 contract

    # Create contract instance
    receipt_erc1155_contract = w3.eth.contract(address=contract_address, abi=receipt_erc1155_abi)

    # Prepare transaction
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = receipt_erc1155_contract.functions.mint(
        recipient_address,
        token_id,
        amount,
        b''  # No data
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 300000,  # Adjust as needed
        'gasPrice': w3.eth.gas_price,
    })
    
    # Sign and send transaction
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    
    # Wait for transaction receipt
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Minting transaction receipt: {tx_receipt}")
    
    if tx_receipt['status'] == 1:
        print(f"Successfully minted {amount} tokens with ID {token_id} to {recipient_address}")
    else:
        print("Minting transaction failed")

    return tx_receipt

def set_token_metadata(contract_address, token_id, name, description, image, terms_of_service, supplemental_images, owner_account):


    # Create contract instance
    receipt_erc1155_contract = w3.eth.contract(address=contract_address, abi=receipt_erc1155_abi)

    # Prepare transaction
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = receipt_erc1155_contract.functions.setTokenMetadata(
        token_id,
        name,
        description,
        image,
        terms_of_service,
        supplemental_images
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 500000,  # Adjust as needed
        'gasPrice': w3.eth.gas_price,
    })
    
    # Sign and send transaction
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    
    # Wait for transaction receipt
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Set metadata transaction receipt: {tx_receipt}")
    
    if tx_receipt['status'] == 1:
        print(f"Successfully set metadata for token ID {token_id}")
    else:
        print("Set metadata transaction failed")

    return tx_receipt

def get_token_metadata(contract_address, token_id):
    # Load the ABI for the ReceiptERC1155 contract


    # Create contract instance
    receipt_erc1155_contract = w3.eth.contract(address=contract_address, abi=receipt_erc1155_abi)

    # Call the getTokenMetadata function
    metadata = receipt_erc1155_contract.functions.getTokenMetadata(token_id).call()
    
    return metadata

def get_token_uri(contract_address, token_id):
    # Load the ABI for the ReceiptERC1155 contract


    # Create contract instance
    receipt_erc1155_contract = w3.eth.contract(address=contract_address, abi=receipt_erc1155_abi)

    # Call the uri function
    uri = receipt_erc1155_contract.functions.uri(token_id).call()
    
    return uri

#set the metadata for the token
token_id = 1

set_metadata_receipt = set_token_metadata(
    new_erc1155_address,
    token_id,
    "Wassie Ghost Pepper Hot Sauce",
    "This is a receipt for item #1",
    "ipfs://bafybeiftsbbjad467epl3ayjpa6z5jeunopbkcp7fw2zk75kykl36sboau", # main image
    "Test Item",
    ["ipfs://bafybeiftsbbjad467epl3ayjpa6z5jeunopbkcp7fw2zk75kykl36sboau"], # supplemental images
    owner_account
)
time.sleep(5)
# mint the NFTs
amount = 100
recipient_address = owner_account.address  # Minting to the owner's address, change if needed



Set metadata transaction receipt: AttributeDict({'blockHash': HexBytes('0x7a2bc30c3b7ae91998c249519e304a9f8213838f85fc98f7072a4b5d846708e6'), 'blockNumber': 27247006, 'contractAddress': None, 'cumulativeGasUsed': 56563419, 'effectiveGasPrice': 2432772, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 300098, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0x437ef56d', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x471cef9af6', 'l1GasPrice': '0x93485ea3', 'l1GasUsed': '0xeda', 'logs': [], 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

TODO: Figure out why the NFT image isn't displaying properly on OpenSea 

In [16]:

#mint_receipt = mint_erc1155_tokens(new_erc1155_address, token_id, amount, recipient_address, owner_account)
token_id =2
time.sleep(5)

set_metadata_receipt = set_token_metadata(
    new_erc1155_address,
    token_id,
    "Solid Gold Wassie Sculpture",
    "This wassie sculpture is made from 1 kg of solid gold",
    "https://ipfs.io/ipfs/bafybeickpw6e6mdniwkk3tkrugsn5h5jroj6gkeq33xs55msm3aaszu4he", # main image
    "Test item",
    ["https://ipfs.io/ipfs/bafybeickpw6e6mdniwkk3tkrugsn5h5jroj6gkeq33xs55msm3aaszu4he"], # supplemental images
    owner_account
)
time.sleep(20)
# mint the NFTs
amount = 100
recipient_address = owner_account.address  # Minting to the owner's address, change if needed

mint_receipt = mint_erc1155_tokens(new_erc1155_address, token_id, amount, recipient_address, owner_account)
  

Set metadata transaction receipt: AttributeDict({'blockHash': HexBytes('0xd3ea84931e2a977b6efe0c9f24c1135f351f918d6f5b6aa56994ffff2f8e421c'), 'blockNumber': 27247345, 'contractAddress': None, 'cumulativeGasUsed': 47458576, 'effectiveGasPrice': 2248833, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 67046, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0x3bff12fd', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x455dfc9018', 'l1GasPrice': '0x86ed1eab', 'l1GasUsed': '0x1043', 'logs': [], 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

# Create Storefront contract using Storefront Factory

## Transfer ERC-1155s to be sold to storefront

(Not included in current user flow, but we will need a frontend to allow sellers to mint ERC-1155 with an image of the item and metadata describing terms of sale will need to go somewhere. Not sure if in the ERC-1155 or in the storefront.)

In [17]:
def create_storefront(factory_contract, owner_account, designated_escrow_agent, erc1155_token, escrow_factory, affiliate_verifier, initial_settle_deadline):
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = factory_contract.functions.createStorefront(
        designated_escrow_agent,
        erc1155_token,
        escrow_factory,
        affiliate_verifier,
        initial_settle_deadline
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 7000000, #used 5.1M gas in testing
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Transaction receipt: {tx_receipt}")
    
    # Process logs to get the new storefront address
    for log in tx_receipt['logs']:
        try:
            event = factory_contract.events.StorefrontCreated().process_log(log)
            print(event)
            if event:
                return event['args']['storefront']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No StorefrontCreated event found in the transaction logs")
    return tx_receipt

# Usage example:
initial_settle_deadline = 3 * 7 * 24 * 60 * 60  # 3 weeks in seconds

new_storefront = create_storefront(
    storefront_factory_contract,
    owner_account,
    owner_account.address,  # designated_escrow_agent
    new_erc1155_address,
    AFFILIATE_ESCROW_FACTORY_ADDRESS,
    AFFILIATE_VERIFIER_PROXY_ADDRESS,
    initial_settle_deadline
)

print(f"New storefront created at: {new_storefront}")

Transaction receipt: AttributeDict({'blockHash': HexBytes('0xf3ff92655724ae6ea52107b4ac71992d7b48563f899bb0a315cb59b108c4f58f'), 'blockNumber': 27247403, 'contractAddress': None, 'cumulativeGasUsed': 45760934, 'effectiveGasPrice': 2243724, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 2363169, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0x2883dbf1', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x1ac8ae5241', 'l1GasPrice': '0x6f4d45d7', 'l1GasUsed': '0x927', 'logs': [AttributeDict({'address': '0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9', 'blockHash': HexBytes('0xf3ff92655724ae6ea52107b4ac71992d7b48563f899bb0a315cb59b108c4f58f'), 'blockNumber': 27247403, 'data': HexBytes('0x'), 'logIndex': 879, 'removed': False, 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x00000000000000000000000059321b6b3aa8573dd33a6314cb48c6849cd55c72')], 'transa

## Seller transfers receipt NFTs to the storefront contract

In [18]:

def transfer_erc1155(erc1155_contract_address, from_account, to_address, token_id, amount):
    nonce = w3.eth.get_transaction_count(from_account.address)

    # Create contract instance
    erc1155_contract = w3.eth.contract(address=erc1155_contract_address, abi=receipt_erc1155_abi)

    txn = erc1155_contract.functions.safeTransferFrom(
        from_account.address,
        to_address,
        token_id,
        amount,
        b''  # data
    ).build_transaction({
        'chainId': w3.eth.chain_id,
        'gas': 200000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_txn = from_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Transferred {amount} of token ID {token_id} to the storefront. Transaction hash: {tx_hash.hex()}")
# Transfer tokens to the storefront
#transfer_erc1155(new_erc1155_address, owner_account, new_storefront, 1, 5)
time.sleep(5)
transfer_erc1155(new_erc1155_address, owner_account, new_storefront, 2, 8)



Transferred 8 of token ID 2 to the storefront. Transaction hash: 0x298d662d08abc1e51f5f19bf82d40579e5f1b775c3fee6361e416ec479aeb9ed


Storefront owner lists the tokens for sale

In [19]:
def list_token(storefront_contract_address, token_id, price, payment_token, affiliate_fee=1000):  # 1000 = 10%
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    # Create contract instance with the affiliate storefront ABI
    storefront_contract = w3.eth.contract(
        address=storefront_contract_address, 
        abi=storefront_abi  # Make sure this is the AffiliateERC1155Storefront ABI
    )
    
    txn = storefront_contract.functions.listToken(
        token_id,
        price,
        payment_token,
        affiliate_fee  # New parameter for affiliate fee in basis points
    ).build_transaction({
        'chainId': w3.eth.chain_id,
        'gas': 200000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Listed token ID {token_id} for sale with {affiliate_fee/100}% affiliate fee.")
    print(f"Transaction hash: {tx_hash.hex()}")
    return tx_receipt

# Example usage:
# ERC20 token address
erc20_address = '0x4ed4E862860beD51a9570b96d89aF5E1B0Efefed'  # $DEGEN on Base

# List tokens for sale with 10% affiliate fee
list_token(
    new_storefront, 
    1, 
    w3.to_wei(1000, 'wei'), 
    erc20_address,
    1000  # 10% affiliate fee
)
time.sleep(5)

list_token(
    new_storefront, 
    2, 
    w3.to_wei(1000, 'wei'), 
    '0x0000000000000000000000000000000000000000',  # Zero address for native token
    1000  # 10% affiliate fee
)

Listed token ID 1 for sale with 10.0% affiliate fee.
Transaction hash: 0x60bc643d5ffe709c12be64dd823214fbb387cd1bddc3c226dd6840ddc1e7e872
Listed token ID 2 for sale with 10.0% affiliate fee.
Transaction hash: 0xe067f1086b34fd66a614cfa545ca9c4b96ca118ac4c0cc2c273a0a990dcae7c6


AttributeDict({'blockHash': HexBytes('0x6b489fda8c8a6e20a4ab5fd334a283afdaeee8b4872c0e67326eaded314efc78'),
 'blockNumber': 27256528,
 'contractAddress': None,
 'cumulativeGasUsed': 41056440,
 'effectiveGasPrice': 1994701,
 'from': '0x9f4640d04371ff6b7886ade5323746388107723a',
 'gasUsed': 194767,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0xb2',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0xa9f12fec',
 'l1GasPrice': '0x2ecf8025',
 'l1GasUsed': '0x640',
 'logs': [AttributeDict({'address': '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF',
   'blockHash': HexBytes('0x6b489fda8c8a6e20a4ab5fd334a283afdaeee8b4872c0e67326eaded314efc78'),
   'blockNumber': 27256528,
   'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001'),
   'logIndex': 393,
   'removed': False,
   'topics': [HexBytes('0x17307eab39ab6107e8899845ad3d59bd9653f200f220920489ca2b5937696c31'),
    HexBytes('0x000000000000000000000000472375ba05fdc78caf9dd9d90a317c82504db1d9'),
    HexBytes('0x000

Storefront owner lowers the price

In [20]:
def check_listing(storefront_contract_address, token_id):
    storefront_contract = w3.eth.contract(address=storefront_contract_address, abi=storefront_abi)
    try:
        listing = storefront_contract.functions.listings(token_id).call()
        if listing[3] == 0:  # listingTime is 0, meaning the token is not listed
            return None
            
        return {
            'tokenId': listing[0],
            'price': listing[1],
            'paymentToken': listing[2],
            'listingTime': listing[3],
            'affiliateFee': listing[4]  # New field for affiliate fee
        }
    except Exception as e:
        print(f"Error checking listing: {str(e)}")
        return None

def update_listing(storefront_contract_address, token_id, new_price, new_affiliate_fee=None):
    storefront_contract = w3.eth.contract(address=storefront_contract_address, abi=storefront_abi)
    try:
        listing = check_listing(storefront_contract_address, token_id)
        if not listing:
            print(f"Token ID {token_id} is not listed.")
            return None
            
        # If no new affiliate fee provided, keep the existing one
        affiliate_fee = new_affiliate_fee if new_affiliate_fee is not None else listing['affiliateFee']
        
        # Validate affiliate fee
        if not 0 <= affiliate_fee <= 10000:
            raise ValueError("Affiliate fee must be between 0 and 10000 basis points (0-100%)")
            
        nonce = w3.eth.get_transaction_count(owner_account.address)
        txn = storefront_contract.functions.updateListing(
            token_id,
            new_price,
            listing['paymentToken'],  # Keep the same payment token
            affiliate_fee
        ).build_transaction({
            'chainId': w3.eth.chain_id,
            'gas': 200000,
            'gasPrice': w3.eth.gas_price,
            'nonce': nonce,
        })
        
        signed_txn = owner_account.sign_transaction(txn)
        tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        
        print(f"Updated listing for token ID {token_id} with price {new_price} and affiliate fee {affiliate_fee/100}%")
        print(f"Transaction hash: {tx_hash.hex()}")
        return tx_receipt
        
    except Exception as e:
        print(f"Error updating listing: {str(e)}")
        return None
# Example usage:
print('Old Listing:')
old_listing = check_listing(new_storefront, 2)
print(old_listing)

print('\nUpdating Listing...')
# Update price and keep same affiliate fee
update_listing(new_storefront, 2, w3.to_wei(2, 'wei'))

print('\nNew Listing:')
new_listing = check_listing(new_storefront, 2)
print(new_listing)

# Example of updating both price and affiliate fee
print('\nUpdating with new affiliate fee...')
update_listing(new_storefront, 2, w3.to_wei(1000, 'wei'), 1500)  # Update to 15% fee


Old Listing:
{'tokenId': 2, 'price': 1000, 'paymentToken': '0x0000000000000000000000000000000000000000', 'listingTime': 1741302403, 'affiliateFee': 1000}

Updating Listing...
Updated listing for token ID 2 with price 2 and affiliate fee 10.0%
Transaction hash: 0x66acafa855eaa1ce7d9c5a75c53f88f387786da6259e01b648a9cc2877884679

New Listing:
{'tokenId': 2, 'price': 2, 'paymentToken': '0x0000000000000000000000000000000000000000', 'listingTime': 1741302421, 'affiliateFee': 1000}

Updating with new affiliate fee...
Updated listing for token ID 2 with price 1000 and affiliate fee 15.0%
Transaction hash: 0x633fe788fc3381f7a46e2755276e99fa08a51d6d12379a9cf1e63890b2df8e85


AttributeDict({'blockHash': HexBytes('0x1668dccdce2d25c269115ecf491d7fab834a0b71971b3203a4427a4e94d55357'),
 'blockNumber': 27256540,
 'contractAddress': None,
 'cumulativeGasUsed': 111342253,
 'effectiveGasPrice': 1995818,
 'from': '0x9f4640d04371ff6b7886ade5323746388107723a',
 'gasUsed': 47444,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0xb2',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0xb10fef79',
 'l1GasPrice': '0x30c5946a',
 'l1GasUsed': '0x640',
 'logs': [AttributeDict({'address': '0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9',
   'blockHash': HexBytes('0x1668dccdce2d25c269115ecf491d7fab834a0b71971b3203a4427a4e94d55357'),
   'blockNumber': 27256540,
   'data': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000003e8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003e800000000000000000000000000000000000000000000000000

Storefront owner toggles the shop on

In [21]:
def set_encryption_public_key(storefront_address, new_key, owner_account):
    """
    Sets the encryption public key for the storefront
    
    Args:
        storefront_address (str): The address of the storefront contract
        new_key (str): The new encryption public key to set
        owner_account: The account object containing the private key of the storefront owner
    """
    try:
        # Create contract instance
        storefront_contract = w3.eth.contract(address=storefront_address, abi=storefront_abi)
        
        # Build transaction
        nonce = w3.eth.get_transaction_count(owner_account.address)
        
        txn = storefront_contract.functions.setEncryptionPublicKey(new_key).build_transaction({
            'chainId': w3.eth.chain_id,
            'gas': 200000,  # Adjust as needed
            'gasPrice': w3.eth.gas_price,
            'nonce': nonce,
        })
        
        # Sign and send transaction
        signed_txn = owner_account.sign_transaction(txn)
        tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
        
        # Wait for transaction receipt
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        
        if tx_receipt['status'] == 1:
            print(f"Successfully set encryption public key. Transaction hash: {tx_hash.hex()}")
        else:
            print("Transaction failed.")
        
        return tx_receipt
        
    except Exception as e:
        print(f"Error setting encryption public key: {str(e)}")
        return None

# Example usage:
new_key = "04853c58dd73f561c1cd0b8372751906d7ed781538e2e4da3108fb2bef871d5d2962784b25446d98d65103d5a9b1761931ecc7a948ffee292aba6eb2a7deb9116a"
tx_receipt = set_encryption_public_key(new_storefront, new_key, owner_account)

Successfully set encryption public key. Transaction hash: 0xf8823a4b4aa667414f95f5e8140a21a1232911cd1cc3f73341ea0ebdaf1f4a45


In [22]:
def get_encryption_key(web3: Web3, storefront_address: str, storefront_abi) -> str:
    """
    Gets the encryption public key from a SimpleERC1155Storefront contract.
    
    Args:
        web3: Web3 instance connected to the appropriate network
        storefront_address: Address of the SimpleERC1155Storefront contract
        storefront_abi: The ABI for the storefront contract
    
    Returns:
        str: The encryption public key string
        
    Raises:
        ValueError: If the contract cannot be loaded or key cannot be retrieved
    """
    try:
        storefront_contract = web3.eth.contract(
            address=web3.to_checksum_address(storefront_address),
            abi=storefront_abi
        )
        encryption_key = storefront_contract.functions.encryptionPublicKey().call()
        
        if not encryption_key:
            raise ValueError("No encryption key set in storefront contract")
            
        return encryption_key
        
    except Exception as e:
        raise ValueError(f"Error getting encryption key: {str(e)}")
#get the encryption key
encryption_key = get_encryption_key(w3, new_storefront, storefront_abi)


In [23]:
def toggle_ready(storefront_contract_address):

    storefront_contract = w3.eth.contract(address=storefront_contract_address, abi=storefront_abi)

    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = storefront_contract.functions.toggleReady().build_transaction({
        'chainId': w3.eth.chain_id,
        'gas': 100000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Toggled ready state. Transaction hash: {tx_hash.hex()}")

# Toggle ready state
toggle_ready(new_storefront)

Toggled ready state. Transaction hash: 0x764396c9d913442ff146a638ce5ab344ae9c26adf2f19cd1f6dee707c3c1fd5a


## Buyer flow

First the buyer checks the listings and fetches the price from the storefront.

TODO: Figure out the best way for the frontend to keep track of what token IDs to check for the listing


In [24]:
from eth_account import Account

# Set up the buyer account
BUYER_PRIVATE_KEY = os.getenv('BUYER_PRIVATE_KEY')
buyer_account = Account.from_key(BUYER_PRIVATE_KEY)
buyer_address = buyer_account.address

# Function to check listing
listing2 = check_listing(new_storefront,2)
print(listing2)


{'tokenId': 2, 'price': 1000, 'paymentToken': '0x0000000000000000000000000000000000000000', 'listingTime': 1741302427, 'affiliateFee': 1500}


In [25]:
new_storefront

'0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9'

The Storefront contract has a hardcoded seaport address.
The buyer needs to get it and the ERC1155 address. 
Then the buyer needs to check if the storefront balance for the token they want to buy is > 0.

In [26]:
def get_seaport_address(storefront_contract_address):

    storefront_contract = w3.eth.contract(address=storefront_contract_address, abi=storefront_abi)

    try:
        return storefront_contract.functions.SEAPORT().call()
    except Exception as e:
        print(f"Error getting Seaport address: {str(e)}")
        return None
    
def get_erc1155_address(storefront_contract_address):
    try:
        storefront_contract = w3.eth.contract(address=storefront_contract_address, abi=storefront_abi)
        return storefront_contract.functions.erc1155Token().call()
    except Exception as e:
        print(f"Error getting ERC-1155 token address: {str(e)}")
        return None

seaport_address = get_seaport_address(new_storefront)
# get the ERC1155 token address
storefront_erc1155_address = get_erc1155_address(new_storefront)

# check the balance of the storefront for the ERC1155 token

def check_storefront_balance(storefront_contract_address, erc1155_token_address, token_id):
    try:
        erc1155_contract = w3.eth.contract(address=erc1155_token_address, abi=receipt_erc1155_abi)
        balance = erc1155_contract.functions.balanceOf(storefront_contract_address, token_id).call()
        return balance
    except Exception as e:
        print(f"Error checking storefront balance: {str(e)}")
        return None
    
balance = check_storefront_balance(new_storefront, storefront_erc1155_address, 2)
print(f"Storefront balance for token ID 2: {balance}")

''' ONLY USED FOR ERC20 TOKEN-DENOMINATED LISTINGS

# finally the user needs to check if seaport has a high enough approval balance to buy the ERC-1155 token
erc20_contract = w3.eth.contract(address=listing2['paymentToken'], abi=erc20_abi)
allowance = erc20_contract.functions.allowance(buyer_address, seaport_address).call()

if allowance < listing2['price']:
    print("Seaport does not have a high enough allowance to buy the token.")
    # approve the seaport contract to spend the payment token
else:
    print("Seaport has a high enough allowance to buy the token.")

'''

Storefront balance for token ID 2: 8


' ONLY USED FOR ERC20 TOKEN-DENOMINATED LISTINGS\n\n# finally the user needs to check if seaport has a high enough approval balance to buy the ERC-1155 token\nerc20_contract = w3.eth.contract(address=listing2[\'paymentToken\'], abi=erc20_abi)\nallowance = erc20_contract.functions.allowance(buyer_address, seaport_address).call()\n\nif allowance < listing2[\'price\']:\n    print("Seaport does not have a high enough allowance to buy the token.")\n    # approve the seaport contract to spend the payment token\nelse:\n    print("Seaport has a high enough allowance to buy the token.")\n\n'

Now buyer calls previewOrder to ensure the trade should work.

In [27]:
# Function to preview order
def preview_order(storefront_contract_address, buyer_address, token_id):

    storefront_contract = w3.eth.contract(address=storefront_contract_address, abi=storefront_abi)

    try:
        listing = check_listing(storefront_contract_address, token_id)
        if not listing:
            print(f"Token ID {token_id} is not listed.")
            return None, None

        erc1155_address = get_erc1155_address(storefront_contract_address) # sorry, we threw coding conventions out the window several cells ago
        if not erc1155_address:
            print("Failed to get ERC-1155 token address.")
            return None, None

        offer = [{
            'itemType': 3,  # ERC1155
            'token': erc1155_address,
            'identifier': token_id,
            'amount': 1  # Assuming we're always buying 1 token
        }]

        consideration = [{
            'itemType': 1 if listing['paymentToken'] != "0x0000000000000000000000000000000000000000" else 0,  # ERC20 or NATIVE
            'token': listing['paymentToken'],
            'identifier': 0,
            'amount': listing['price']
        }]
        print(offer)
        print(consideration)
        preview_result = storefront_contract.functions.previewOrder(
            buyer_address,
            buyer_address,
            offer,
            consideration,
            "0x"  # Empty bytes for unused parameter
        ).call()

        return preview_result
    except Exception as e:
        print(f"Error in preview_order: {str(e)}")
        return None, None
#get listing 1
listing2 = check_listing(new_storefront, 2)
print(listing2)
offer,consideration = preview_order(new_storefront, buyer_address, listing2['tokenId'])
print(offer)
print(consideration)

{'tokenId': 2, 'price': 1000, 'paymentToken': '0x0000000000000000000000000000000000000000', 'listingTime': 1741302427, 'affiliateFee': 1500}
[{'itemType': 3, 'token': '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 'identifier': 2, 'amount': 1}]
[{'itemType': 0, 'token': '0x0000000000000000000000000000000000000000', 'identifier': 0, 'amount': 1000}]
[(3, '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 2, 1)]
[(0, '0x0000000000000000000000000000000000000000', 0, 1000, '0x2Ad2b3309373bf95ab47C40Ac773f0c70536Cc85')]


previewOrder returns the offer (what is being sold by the storefront) and the consideration (what the storefront is willing to accept for the offer). 
We pretty much just pass this to fulfill_order

In [28]:
from eth_keys import keys
from eth_utils import keccak, to_bytes, to_hex
from Crypto.Cipher import AES
import os
import struct
from typing import Optional, Tuple, Union

class EthereumMessageEncryption:
    BLOCK_SIZES = [64, 128, 256, 512, 1024, 2048]  # Same block sizes as JS version
    
    def __init__(self, web3):
        self.web3 = web3
    
    def strip_hex_prefix(self, hex_str: str) -> str:
        """Remove 0x prefix from hex string if present."""
        return hex_str[2:] if hex_str.startswith('0x') else hex_str
    
    def order_public_keys(self, key1: str, key2: str) -> Tuple[str, str]:
        """Order two public keys lexicographically."""
        clean1 = self.strip_hex_prefix(key1)
        clean2 = self.strip_hex_prefix(key2)
        return (key1, key2) if clean1 < clean2 else (key2, key1)
    
    def derive_shared_secret(self, public_key1: str, public_key2: str) -> bytes:
        """Derive a shared secret from two public keys."""
        first_key, second_key = self.order_public_keys(public_key1, public_key2)
        combined = bytes.fromhex(self.strip_hex_prefix(first_key)) + bytes.fromhex(self.strip_hex_prefix(second_key))
        return keccak(combined)
    
    def get_target_block_size(self, message_length: int) -> int:
        """Get the appropriate block size for a message of given length."""
        for size in self.BLOCK_SIZES:
            if message_length <= size - 9:  # 9 bytes for length prefix
                return size
        raise ValueError(f"Message too long. Maximum size is {self.BLOCK_SIZES[-1] - 9} bytes")
    
    def pad_message(self, message_bytes: bytes) -> bytes:
        """Pad message according to the same scheme as JS version."""
        original_length = len(message_bytes)
        target_size = self.get_target_block_size(original_length)
        
        # Create length bytes (8 bytes, little endian)
        length_bytes = struct.pack('<Q', original_length)
        
        # Create padded message
        padded_message = bytearray(target_size)
        padded_message[0] = 8  # Length of length field
        padded_message[1:9] = length_bytes
        padded_message[9:9 + original_length] = message_bytes
        
        # Fill remaining space with random bytes
        padding = os.urandom(target_size - original_length - 9)
        padded_message[9 + original_length:] = padding
        
        return bytes(padded_message)
    
    def unpad_message(self, padded_message: bytes) -> bytes:
        """Unpad message according to the same scheme as JS version."""
        length_bytes_count = padded_message[0]
        length = struct.unpack('<Q', padded_message[1:1 + length_bytes_count])[0]
        return padded_message[9:9 + length]
    
    def create_message_hash(self, message: bytes, salt: bytes) -> str:
        """Create a hash of the message with salt."""
        salted_message = message + salt
        return to_hex(keccak(salted_message))
    
    def encrypt_message(self, message: str, recipient_public_key: str, include_verification: bool = False) -> dict:
        """
        Encrypt a message for a recipient using their public key.
        
        Args:
            message: The message to encrypt
            recipient_public_key: Recipient's public key
            include_verification: Whether to include verification hash
            
        Returns:
            dict: Encrypted message data including:
                - encryptedData
                - ephemeralPublicKey
                - iv
                - verificationHash (if verification enabled)
        """
        try:
            # Convert message to bytes
            message_bytes = message.encode('utf-8')
            
            padded_message: bytes
            verification_hash: Optional[str] = None
            salt: Optional[bytes] = None
            
            if include_verification:
                # Generate random salt and hash
                salt = os.urandom(32)
                verification_hash = self.create_message_hash(message_bytes, salt)
                
                # Combine message and salt
                message_with_salt = message_bytes + salt
                padded_message = self.pad_message(message_with_salt)
            else:
                padded_message = self.pad_message(message_bytes)
            
            # Generate ephemeral key pair
            ephemeral_private_key = keys.PrivateKey(os.urandom(32))
            ephemeral_public_key = ephemeral_private_key.public_key
            
            # Derive shared secret
            shared_secret = self.derive_shared_secret(
                recipient_public_key,
                to_hex(ephemeral_public_key.to_bytes())
            )
            
            # Generate IV for AES-GCM
            iv = os.urandom(12)
            
            # Use first 32 bytes of shared secret as key
            key = shared_secret[:32]
            
            # Encrypt using AES-GCM
            cipher = AES.new(key, AES.MODE_GCM, nonce=iv)
            encrypted_data, tag = cipher.encrypt_and_digest(padded_message)
            
            # Combine encrypted data and tag
            full_encrypted = encrypted_data + tag
            
            result = {
                "encryptedData": full_encrypted.hex(),
                "ephemeralPublicKey": ephemeral_public_key.to_hex(),
                "iv": iv.hex(),
                "scheme": "password"
            }
            
            if include_verification:
                result["verificationHash"] = verification_hash
                result["verificationEnabled"] = True
            
            return result
            
        except Exception as e:
            raise Exception(f"Failed to encrypt message: {str(e)}")

    def verify_message(self, message: str, salt: bytes, hash: str) -> bool:
        """Verify a decrypted message matches its hash."""
        message_bytes = message.encode('utf-8')
        computed_hash = self.create_message_hash(message_bytes, salt)
        return computed_hash == hash
    
encryption = EthereumMessageEncryption(w3)

# Get recipient's public key
recipient_key = get_encryption_key(w3, new_storefront, storefront_abi)

# Encrypt a message
encrypted = encryption.encrypt_message(
    "test", 
    recipient_key,
    include_verification=False
)
    

Verified that the above encrypted data decrypted using https://ipfs.io/ipfs/bafybeigjvecrlmlthwijbtcp62v2hy4fvwivcweksoyfrzcqjbjzzihtwi/
source:
https://github.com/nickbax/hermes/blob/main/src/components/MessageEncryptionUI.tsx
https://github.com/nickbax/hermes/blob/main/src/lib/EthereumMessageEncryption.ts


In [29]:
import traceback
from eth_abi import encode
from web3 import Web3

def fulfill_advanced_order(storefront_address, buyer_address, token_id, offer, consideration):
    try:
        # Very minimal test message - convert hex strings to bytes
        encrypted_message = {
            "encryptedData": bytes.fromhex("b31bf3a238ffcdc022697f3599f16211c23994b50dd90ac3abdb74a32140741003defa93bb064827aa165b8972328214b40440375b754a4a85bb6ffdd9fce13e36c5915ff5b5f8af8e2bfb6e90eb8815"),
            "ephemeralPublicKey": bytes.fromhex("046901a52162ee78d8f14c43c1e29c228211fb416e73d212a1b9608ee5d97d21d3440782ddbc07b88985d2f54c9b055876b110a5c8c6b98f5ff7be640397f2032e"),
            "iv": bytes.fromhex("882b87d42312fa211e1bc9df"),
            "verificationHash": b'' # Empty bytes for no verification hash
        }

        # First encode the affiliate address separately
        affiliate_address = "0x9B7A78f673678b247a3808496786dd16C61Cdb32"
        encoded_affiliate = encode(['address'], [Web3.to_checksum_address(affiliate_address)])

        # Then encode the message components separately
        encoded_message = encode(
            ['bytes', 'bytes', 'bytes', 'bytes'],  # Separate types, not a tuple
            [
                encrypted_message['encryptedData'],
                encrypted_message['ephemeralPublicKey'],
                encrypted_message['iv'],
                encrypted_message['verificationHash']
            ]
        )

        # Combine the encoded parts
        context_data = encoded_affiliate + encoded_message

        order_parameters = (
            storefront_address,
            "0x0000000000000000000000000000000000000000",
            offer,
            consideration,
            4, # CONTRACT order type
            0,
            2**256 - 1,
            "0x0000000000000000000000000000000000000000000000000000000000000000",
            0,
            "0x0000000000000000000000000000000000000000000000000000000000000000",
            len(consideration)
        )

        advanced_order = (
            order_parameters,
            1, # numerator
            1, # denominator
            Web3.to_bytes(hexstr="0x"), # signature
            context_data
        )

        seaport_contract = w3.eth.contract(
            address=get_seaport_address(storefront_address),
            abi=seaport_abi
        )

        is_native_payment = consideration[0][1] == '0x0000000000000000000000000000000000000000'
        native_value = int(consideration[0][3]) if is_native_payment else 0

        tx = seaport_contract.functions.fulfillAdvancedOrder(
            advanced_order,
            [], # criteriaResolvers
            "0x0000000000000000000000000000000000000000000000000000000000000000", # fulfillerConduitKey
            buyer_address
        ).build_transaction({
            'from': buyer_address,
            'gas': 3000000,
            'gasPrice': w3.eth.gas_price,
            'nonce': w3.eth.get_transaction_count(buyer_address),
            'value': native_value
        })

        print("Context data hexstring:", context_data.hex()) # Debug print
        print("Offer:", offer)
        print("Consideration:", consideration)

        signed_tx = buyer_account.sign_transaction(tx)
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Order fulfillment successful. Hash: {tx_receipt.transactionHash.hex()}")
        return tx_receipt

    except Exception as e:
        print(f"Error in fulfill_advanced_order: {str(e)}")
        traceback.print_exc()
        return None


In [31]:

# Usage example:
token_id = 2 # The token ID we want to buy
offer, consideration = preview_order(new_storefront, buyer_address, token_id)
if offer and consideration:
    print("Offer:", offer)
    print("Consideration:", consideration)
    # Adjust the offer and consideration arrays for the required format
    adjusted_offer = [(item[0], item[1], item[2], item[3], item[3]) for item in offer]
    adjusted_consideration = [(item[0], item[1], item[2], item[3], item[3], item[4]) for item in consideration]
    
    print("Fulfilling the order...")
    tx_receipt = fulfill_advanced_order(
        new_storefront,
        buyer_address,
        token_id,
        adjusted_offer,
        adjusted_consideration
    )
    if tx_receipt:
        print("Order fulfilled successfully!")
    else:
        print("Order fulfillment failed")
else:
    print("Failed to preview order")

[{'itemType': 3, 'token': '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 'identifier': 2, 'amount': 1}]
[{'itemType': 0, 'token': '0x0000000000000000000000000000000000000000', 'identifier': 0, 'amount': 1000}]
Offer: [(3, '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 2, 1)]
Consideration: [(0, '0x0000000000000000000000000000000000000000', 0, 1000, '0x2Ad2b3309373bf95ab47C40Ac773f0c70536Cc85')]
Fulfilling the order...
Context data hexstring: 0000000000000000000000009b7a78f673678b247a3808496786dd16c61cdb3200000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000018000000000000000000000000000000000000000000000000000000000000001c00000000000000000000000000000000000000000000000000000000000000050b31bf3a238ffcdc022697f3599f16211c23994b50dd90ac3abdb74a32140741003defa93bb064827aa165b8972328214b40440375b754a4a85bb6ffdd9fce13e36c5915ff5b5f8af8e2bfb6e90eb8815000000000000

Traceback (most recent call last):
  File "/tmp/ipykernel_2299888/643991113.py", line 81, in fulfill_advanced_order
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
  File "/home/ojo/.local/lib/python3.9/site-packages/web3/eth/eth.py", line 396, in send_raw_transaction
    return self._send_raw_transaction(transaction)
  File "/home/ojo/.local/lib/python3.9/site-packages/web3/module.py", line 75, in caller
    result = w3.manager.request_blocking(
  File "/home/ojo/.local/lib/python3.9/site-packages/web3/manager.py", line 330, in request_blocking
    return self.formatted_response(
  File "/home/ojo/.local/lib/python3.9/site-packages/web3/manager.py", line 293, in formatted_response
    raise ValueError(error)
ValueError: {'code': -32000, 'message': 'replacement transaction underpriced'}


Let's test settling the affiliate escrow (until the graph is working, we input the escrow contract address manually)

In [9]:
escrow_address = '0x2245Bab7A4183e303c7BaA7cb374D65d41cd499f'

# First create the contract instance
escrow_contract = w3.eth.contract(address=escrow_address, abi=affiliate_escrow_abi)

def settle_escrow(escrow_address, escrow_contract, account):
    balance = w3.eth.get_balance(escrow_address)
    
    # Build the transaction
    nonce = w3.eth.get_transaction_count(account.address)
    
    settle_txn = escrow_contract.functions.settle(
        '0x0000000000000000000000000000000000000000',  # ETH address
        balance
    ).build_transaction({
        'from': account.address,
        'gas': 200000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    # Sign and send the transaction
    signed_txn = account.sign_transaction(settle_txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    if tx_receipt['status'] == 1:
        print(f"Settlement successful! Transaction hash: {tx_hash.hex()}")
    else:
        print("Settlement failed!")
    
    return tx_receipt

# Now call the function with the correct parameters
settle_escrow(escrow_address, escrow_contract, buyer_account)

NameError: name 'buyer_account' is not defined

In [109]:
buyer_account.address


'0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B'

Let's test removing the listing and making a new ETH-denominated listing

## Now let's list an item for ETH

### Now buyer takes the order

## Finally, we need to test the escrow functions



In [34]:
import requests
# Replace with your actual subgraph URL
SUBGRAPH_URL = "https://api.studio.thegraph.com/query/90920/ump-affiliate-devtest/version/latest"

def query_subgraph(query, variables=None):
    request = requests.post(SUBGRAPH_URL, json={'query': query, 'variables': variables})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception(f"Query failed. Return code is {request.status_code}. {query}")

# Get all escrow events for user address
user_address = "0x9f4640d04371ff6b7886ade5323746388107723a"

query = """
query GetEscrowData($userAddress: Bytes!) {
  orderEscrows(
    where: {
      or: [
        {payee: $userAddress},
        {arbiter: $userAddress}
      ]
    }
  ) {
    id
    escrowAddress
    payee
    storefront
    arbiter
    isDisputed
    isRefunded
    blockNumber
    blockTimestamp
    payments {
      id
      payer
      settleDeadline
      blockTimestamp
      transactionHash
      orderFulfilled {
        id
        orderHash
        blockTimestamp
        transactionHash
        offer {
          itemType
          token
          identifier
          amount
        }
        consideration {
          itemType
          token
          identifier
          amount
          recipient
        }
      }
    }
    settledEvents {
      id
      to
      token
      amount
      blockTimestamp
      transactionHash
    }
    refundedEvents {
      id
      to
      token
      amount
      blockTimestamp
      transactionHash
    }
    disputedEvents {
      id
      disputeInitiator
      blockTimestamp
      transactionHash
    }
    disputeRemovedEvents {
      id
      disputeRemover
      blockTimestamp
      transactionHash
    }
    disputeResolvedEvents {
      id
      resolver
      settled
      blockTimestamp
      transactionHash
    }
    escapeAddressSetEvents {
      id
      escapeAddress
      blockNumber
      blockTimestamp
      transactionHash
    }
    escapedEvents {
      id
      to
      token
      amount
      blockTimestamp
      transactionHash
    }
    arbiterChangeEvents {
      id
      oldArbiter
      proposedArbiter
      newArbiter
      approved
      approver
      blockTimestamp
      transactionHash
    }
  }
}
"""

variables = {'userAddress': user_address.lower()}
result = query_subgraph(query, variables)
result


{'data': {'orderEscrows': [{'id': '0x1afdf02b01d4403b29cf0219a89b2b0b37068b35',
    'escrowAddress': '0x1afdf02b01d4403b29cf0219a89b2b0b37068b35',
    'payee': '0x9f4640d04371ff6b7886ade5323746388107723a',
    'storefront': '0xe27bb219cf9c3806fa41bf1e28086e7e41685de4',
    'arbiter': '0x9f4640d04371ff6b7886ade5323746388107723a',
    'isDisputed': False,
    'isRefunded': False,
    'blockNumber': '27215279',
    'blockTimestamp': '1741219905',
    'payments': [],
    'settledEvents': [],
    'refundedEvents': [],
    'disputedEvents': [],
    'disputeRemovedEvents': [],
    'disputeResolvedEvents': [],
    'escapeAddressSetEvents': [],
    'escapedEvents': [],
    'arbiterChangeEvents': []},
   {'id': '0x2245bab7a4183e303c7baa7cb374d65d41cd499f',
    'escrowAddress': '0x2245bab7a4183e303c7baa7cb374d65d41cd499f',
    'payee': '0x9f4640d04371ff6b7886ade5323746388107723a',
    'storefront': '0x72b28ffac4731ff1ce66b5fa456a8517667bf773',
    'arbiter': '0x9f4640d04371ff6b7886ade532374638810

The data shows that our buyer has made 2 transactions and because this is a test, is also the escrowAgent on all of these escrows.

Let's test settling the first transaction.

In [46]:

def check_balance_and_settle(escrow_address, buyer_address, private_key, consideration):
    escrow_contract = w3.eth.contract(address=escrow_address, abi=affiliate_escrow_abi)

    # Extract token information from consideration
    token_address = w3.to_checksum_address(consideration['token'])
    expected_amount = int(consideration['amount'])
    print(consideration)

    # Check balance
    if token_address == '0x0000000000000000000000000000000000000000':
        # ETH
        balance = w3.eth.get_balance(escrow_address)
        token_symbol = 'ETH'
        decimals = 18
    else:
        # ERC20
        token_contract = w3.eth.contract(address=token_address, abi=erc20_abi)
        balance = token_contract.functions.balanceOf(escrow_address).call()
        token_symbol = token_contract.functions.symbol().call()
        decimals = token_contract.functions.decimals().call()

    print(f"Escrow contract balance: {balance / 10**decimals} {token_symbol}")

    if balance < expected_amount:
        print(f"Warning: Balance ({balance}) is less than expected amount ({expected_amount})")

    # Prepare the settle transaction
    nonce = w3.eth.get_transaction_count(buyer_address)
    
    settle_tx = escrow_contract.functions.settle(
        token_address,
        balance  # Use the actual balance
    ).build_transaction({
        'chainId': w3.eth.chain_id,
        'gas': 200000,  # Adjust as necessary
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })

    # Sign the transaction
    signed_tx = w3.eth.account.sign_transaction(settle_tx, private_key)

    # Send the transaction
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

    # Wait for the transaction receipt
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    if tx_receipt['status'] == 1:
        print(f"Settlement successful. Transaction hash: {tx_hash.hex()}")
    else:
        print(f"Settlement failed. Transaction hash: {tx_hash.hex()}")

    return tx_receipt

# escrow address is 0x2Ad2b3309373bf95ab47C40Ac773f0c70536Cc85
escrow_address = '0x2Ad2b3309373bf95ab47C40Ac773f0c70536Cc85'
# Example consideration from escrow_transaction_data/consolidated view
escrow_address = w3.to_checksum_address(result['data']['orderEscrows'][2]['escrowAddress'])

consideration = result['data']['orderEscrows'][2]['payments'][0]['orderFulfilled']['consideration'][0]

tx_receipt = check_balance_and_settle(escrow_address, buyer_address, buyer_account.key, consideration)


{'itemType': '0', 'token': '0x0000000000000000000000000000000000000000', 'identifier': '0', 'amount': '1000', 'recipient': '0x2ad2b3309373bf95ab47c40ac773f0c70536cc85'}
Escrow contract balance: 1e-15 ETH


HTTPError: 429 Client Error: Too Many Requests for url: https://base-mainnet.infura.io/v3/26DtrLheKXrxF7h7gWZ2N2fYUVw

code bugged out with error 429 but here's the tx hash 0x0927cdcfba4b0c3c55fb1d8cb202abb47236cc966e7dc74e53e80c5c141d0c3b

# UNTESTED BELOW HERE
We have a disputed transaction as well in contract 0xCB879F9bc4F72DA93Eb149f7463a0964E4913514

Let's settle this dispute

In [160]:

#right now this is hardcoded to use degen
def settle_dispute(escrow_address, escrow_agent_address, private_key, should_settle, consideration):
    # Create contract instance
    escrow_contract = w3.eth.contract(address=escrow_address, abi=simple_escrow_abi)

    # Extract token information from consideration
    token_address = "0x4ed4E862860beD51a9570b96d89aF5E1B0Efefed"
    amount = 2#consideration['amount']

    # Prepare the resolveDispute transaction
    nonce = w3.eth.get_transaction_count(escrow_agent_address)
    
    resolve_dispute_tx = escrow_contract.functions.resolveDispute(
        should_settle,
        token_address,
        amount
    ).build_transaction({
        'chainId': w3.eth.chain_id,
        'gas': 200000,  # Adjust as necessary
        'gasPrice': w3.eth.gas_price+1,
        'nonce': nonce,
    })

    # Sign the transaction
    signed_tx = w3.eth.account.sign_transaction(resolve_dispute_tx, private_key)

    # Send the transaction
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

    # Wait for the transaction receipt
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    if tx_receipt['status'] == 1:
        result = "settled" if should_settle else "refunded"
        print(f"Dispute resolution successful. The escrow was {result}. Transaction hash: {tx_hash.hex()}")
    else:
        print(f"Dispute resolution failed. Transaction hash: {tx_hash.hex()}")

    return tx_receipt

#note: the buyer account is also the escrow agent in this test case
settle_dispute('0x33f71861ecBFa6b0fecE625F5fFbb66A7281Dd8E', '0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B', buyer_account.key, False, 1)

Dispute resolution successful. The escrow was refunded. Transaction hash: 100602e97a3ed6456400775267aa44dbe4576345d5dda0c89a57de3120a3700a


AttributeDict({'blockHash': HexBytes('0xd6ac5b0271bc7e88688c79a983bac36023f15b3e1a05358f9231df2c784d6d69'),
 'blockNumber': 22230242,
 'contractAddress': None,
 'cumulativeGasUsed': 30304258,
 'effectiveGasPrice': 9320000,
 'from': '0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B',
 'gasUsed': 52436,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0x278aaebf5',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0x28a105afc0',
 'l1FeeScalar': '0.002269',
 'l1GasPrice': '0x7792cd1bc',
 'l1GasUsed': '0x95c',
 'logs': [AttributeDict({'address': '0x4ed4E862860beD51a9570b96d89aF5E1B0Efefed',
   'blockHash': HexBytes('0xd6ac5b0271bc7e88688c79a983bac36023f15b3e1a05358f9231df2c784d6d69'),
   'blockNumber': 22230242,
   'blockTimestamp': '0x6730c6a7',
   'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000002'),
   'logIndex': 674,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000

In [112]:
# create curation storefront
from web3 import Web3
import json
import requests
from eth_account import Account
import os
from dotenv import load_dotenv

# Load environment variables and setup
load_dotenv()
w3 = Web3(Web3.HTTPProvider(os.getenv('RPC_URL')))
SUBGRAPH_URL = "https://api.studio.thegraph.com/query/90920/ump/version/latest"
CURATION_ADDRESS = "0x7b1B48E021cc00eE5e0a4CD57d23eE2b922078c3"

# ABI for CurationStorefront - this is a minimal version with just the functions we need
CURATION_ABI = [
    {
        "inputs": [
            {"internalType": "string", "name": "name", "type": "string"},
            {"internalType": "string", "name": "description", "type": "string"},
            {"internalType": "address", "name": "paymentAddress", "type": "address"},
            {"internalType": "string", "name": "tokenURI", "type": "string"}
        ],
        "name": "createCuration",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "nonpayable",
        "type": "function"
    }
]

def create_curation(name, description, payment_address, token_uri, private_key):
    account = Account.from_key(private_key)
    contract = w3.eth.contract(address=CURATION_ADDRESS, abi=CURATION_ABI)
    
    nonce = w3.eth.get_transaction_count(account.address)
    
    transaction = contract.functions.createCuration(
        name,
        description,
        payment_address,
        token_uri
    ).build_transaction({
        'from': account.address,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_txn = account.sign_transaction(transaction)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    return receipt

def query_active_listings():
    query = """
    query {
      storefronts {
        id
        storefrontAddress
        owner
        erc1155Token
        ready
        listings(where: { active: true }) {
          id
          tokenId
          price
          paymentToken
          listingTime
          tokenURI
          tokenMetadata {
            rawJson
          }
        }
      }
    }
    """
    
    try:
        response = requests.post(
            SUBGRAPH_URL,
            json={'query': query}
        )
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Query failed with status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error querying subgraph: {str(e)}")
        return None

# Example usage:
def create_example_curation():
    # Replace with your private key and payment address
    private_key = os.getenv('PRIVATE_KEY')
    payment_address = Account.from_key(private_key).address
    
    name = "Example Curation"
    description = "This is an example curation collection"
    token_uri = "ipfs://example/metadata.json"
    
    receipt = create_curation(
        name,
        description,
        payment_address,
        token_uri,
        private_key
    )
    
    print(f"Curation created in transaction: {receipt.transactionHash.hex()}")
    return receipt

def print_active_listings():
    results = query_active_listings()
    if not results or 'data' not in results:
        print("No data returned from query")
        return
    
    storefronts = results['data']['storefronts']
    for storefront in storefronts:
        print(f"\nStorefront: {storefront['storefrontAddress']}")
        print(f"Owner: {storefront['owner']}")
        print(f"ERC1155 Token: {storefront['erc1155Token']}")
        print(f"Ready: {storefront['ready']}")
        print("\nActive Listings:")
        
        for listing in storefront['listings']:
            print(f"\nToken ID: {listing['tokenId']}")
            print(f"Price: {listing['price']}")
            print(f"Payment Token: {listing['paymentToken']}")
            print(f"Listing Time: {listing['listingTime']}")
            
            if listing['tokenMetadata'] and listing['tokenMetadata']['rawJson']:
                try:
                    metadata = json.loads(listing['tokenMetadata']['rawJson'])
                    print(f"Token Metadata: {json.dumps(metadata, indent=2)}")
                except:
                    print("Could not parse token metadata")



In [113]:
receipt = create_example_curation()
receipt

Curation created in transaction: 0xdf64a1e957c0464dd0cc32483f11739ffab6d53bc419b8dab8df036d8c72ba1f


AttributeDict({'blockHash': HexBytes('0x3eeca3ad7400d78bf9503e199e4a1f21a9b10d4a45d9e62e17752d5ac0f83094'),
 'blockNumber': 26771104,
 'contractAddress': None,
 'cumulativeGasUsed': 42642091,
 'effectiveGasPrice': 1896745,
 'from': '0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B',
 'gasUsed': 244627,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0x8c33acc',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0x728529b6c',
 'l1GasPrice': '0x2a43f068',
 'l1GasUsed': '0xa9e',
 'logs': [AttributeDict({'address': '0x7b1B48E021cc00eE5e0a4CD57d23eE2b922078c3',
   'blockHash': HexBytes('0x3eeca3ad7400d78bf9503e199e4a1f21a9b10d4a45d9e62e17752d5ac0f83094'),
   'blockNumber': 26771104,
   'data': HexBytes('0x'),
   'logIndex': 514,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x000000000000000000000000e360ee830bb3df8ab17c277f1802a2e861850

In [96]:

print_active_listings()


Storefront: 0x021dc1a5f422b623a78766f53bc7a6936e9a49d0
Owner: 0x7953c361b4495f7c6f4f0d9fc27dec2919fea545
ERC1155 Token: 0x1cea52b89ea507d567fb74baa05d9b83f13a75ab
Ready: False

Active Listings:

Storefront: 0x086eaf091200e3d294fe26bf3c56801fe3866fd9
Owner: 0xa320624b3cc13f83be03ff32fc9fea2d6b491c53
ERC1155 Token: 0xa7fd6673f027fdc841b63fd71a580254d92faf26
Ready: False

Active Listings:

Storefront: 0x0b468e5e8c8df2d875ea83c564ed509f2d8281c1
Owner: 0xefd5d2c7a831981b5e7ebf12226b08ab3097ece0
ERC1155 Token: 0x6c62649755c4d9af00c36ef33d0e5305627991a1
Ready: True

Active Listings:

Token ID: 1
Price: 1000
Payment Token: 0x0000000000000000000000000000000000000000
Listing Time: 1737902341
Could not parse token metadata

Storefront: 0x0d879552537e635f1cf27eb66358230959f06b9b
Owner: 0x9b7a78f673678b247a3808496786dd16c61cdb32
ERC1155 Token: 0x4b49c64e615e085cbfe8578992b2ba1e3f45916b
Ready: False

Active Listings:

Token ID: 1
Price: 350000000000000000000
Payment Token: 0x4ed4e862860bed51a9570b9